### Import required libraries

In [1]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import numpy as np # linear algebra

import warnings
warnings.simplefilter('ignore')

import pickle

### Read in the csv file

In [2]:
df = pd.read_csv('../HistoricalProductDemand.csv')
df.head()

,Product_Code,Warehouse,Product_Category,Date,Order_Demand
0,Product_0993,Whse_J,Category_028,2012/7/27,100
1,Product_0979,Whse_J,Category_028,2012/1/19,500
2,Product_0979,Whse_J,Category_028,2012/2/3,500
3,Product_0979,Whse_J,Category_028,2012/2/9,500
4,Product_0979,Whse_J,Category_028,2012/3/2,500


**The original Columns are:**  

Product_Code - The product name encoded  
Warehouse - Warehouse name encoded  
Product_Category - Product Category for each Product_Code encoded  
Date - The date customer needs the product  
Order_Demand - single order qty  

In [3]:
df.shape

(1048575, 5)

### Explore data characteristics

In [4]:
df.describe()

,Product_Code,Warehouse,Product_Category,Date,Order_Demand
count,1048575,1048575,1048575,1037336,1048575
unique,2160,4,33,1729,3828
top,Product_1359,Whse_J,Category_019,2013/9/27,1000
freq,16936,764447,481099,2075,112682


In [5]:
df.dtypes

Product_Code        object
Warehouse           object
Product_Category    object
Date                object
Order_Demand        object
dtype: object

In [6]:
df[df.Date.isnull()==False]

,Product_Code,Warehouse,Product_Category,Date,Order_Demand
0,Product_0993,Whse_J,Category_028,2012/7/27,100
1,Product_0979,Whse_J,Category_028,2012/1/19,500
2,Product_0979,Whse_J,Category_028,2012/2/3,500
3,Product_0979,Whse_J,Category_028,2012/2/9,500
4,Product_0979,Whse_J,Category_028,2012/3/2,500
...,...,...,...,...,...
1048570,Product_1791,Whse_J,Category_006,2016/4/27,1000
1048571,Product_1974,Whse_J,Category_006,2016/4/27,1
1048572,Product_1787,Whse_J,Category_006,2016/4/28,2500
1048573,Product_0901,Whse_J,Category_023,2016/10/7,50


In [7]:
df[df.Date.isnull()==True]

,Product_Code,Warehouse,Product_Category,Date,Order_Demand
45460,Product_1461,Whse_A,Category_019,NaN,10000
456373,Product_1636,Whse_A,Category_019,NaN,100
456374,Product_1461,Whse_A,Category_019,NaN,300
456375,Product_1464,Whse_A,Category_019,NaN,300
456376,Product_1388,Whse_A,Category_019,NaN,200
...,...,...,...,...,...
995790,Product_1464,Whse_A,Category_019,NaN,(900)
995791,Product_1541,Whse_A,Category_019,NaN,(200)
995792,Product_1388,Whse_A,Category_019,NaN,(300)
995793,Product_1541,Whse_A,Category_019,NaN,(300)


In [8]:
#Notice that the last NaN occurs right before December 12, 2011
#Limit data to 2012 and after??

df[df.index == 99575]

,Product_Code,Warehouse,Product_Category,Date,Order_Demand
99575,Product_0631,Whse_J,Category_001,2012/12/11,10


In [9]:
df[df.Date.isnull()==True].iloc[20:30, :]

,Product_Code,Warehouse,Product_Category,Date,Order_Demand
472017,Product_0423,Whse_A,Category_019,NaN,100
472018,Product_0428,Whse_A,Category_019,NaN,100
472019,Product_1416,Whse_A,Category_019,NaN,200
472020,Product_0414,Whse_A,Category_019,NaN,100
472021,Product_0416,Whse_A,Category_019,NaN,400
472022,Product_0417,Whse_A,Category_019,NaN,400
472023,Product_0423,Whse_A,Category_019,NaN,200
472024,Product_0428,Whse_A,Category_019,NaN,100
472025,Product_1416,Whse_A,Category_019,NaN,200
472026,Product_1424,Whse_A,Category_019,NaN,200


In [10]:
df.columns

Index(['Product_Code', 'Warehouse', 'Product_Category', 'Date',
       'Order_Demand'],
      dtype='object')

In [11]:
products = list(df.Product_Code.unique())
len(products)

2160

In [12]:
wh = list(df.Warehouse.unique())
wh

['Whse_J', 'Whse_S', 'Whse_C', 'Whse_A']

In [13]:
category = list(df.Product_Category.unique())
category

['Category_028',
 'Category_006',
 'Category_001',
 'Category_019',
 'Category_005',
 'Category_030',
 'Category_032',
 'Category_011',
 'Category_033',
 'Category_007',
 'Category_015',
 'Category_021',
 'Category_024',
 'Category_026',
 'Category_023',
 'Category_022',
 'Category_009',
 'Category_003',
 'Category_004',
 'Category_018',
 'Category_020',
 'Category_013',
 'Category_008',
 'Category_017',
 'Category_031',
 'Category_025',
 'Category_010',
 'Category_029',
 'Category_012',
 'Category_002',
 'Category_014',
 'Category_027',
 'Category_016']

In [14]:
#Assuming that parenthesis indicate a return order so changing Order_Demand to negative
df['Order_Demand'] = df['Order_Demand'].map(lambda x: x.replace('(', '-').rstrip(')'))

In [15]:
#Check to make sure no paren in column
df.Order_Demand.apply(lambda x: 1 if ('(' in x) | (')' in x) else 0).sum()

0

In [16]:
#Change Order_Demand to int64
df['Order_Demand'] = df['Order_Demand'].apply(pd.to_numeric)

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 5 columns):
 #   Column            Non-Null Count    Dtype 
---  ------            --------------    ----- 
 0   Product_Code      1048575 non-null  object
 1   Warehouse         1048575 non-null  object
 2   Product_Category  1048575 non-null  object
 3   Date              1037336 non-null  object
 4   Order_Demand      1048575 non-null  int64 
dtypes: int64(1), object(4)
memory usage: 40.0+ MB


In [18]:
df.Date.head()

0    2012/7/27
1    2012/1/19
2     2012/2/3
3     2012/2/9
4     2012/3/2
Name: Date, dtype: object

In [19]:
#Convert Date column to datetime objects

df['Date'] = pd.to_datetime(df.Date, yearfirst = True)

In [20]:
df.Date.head()

0   2012-07-27
1   2012-01-19
2   2012-02-03
3   2012-02-09
4   2012-03-02
Name: Date, dtype: datetime64[ns]

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 5 columns):
 #   Column            Non-Null Count    Dtype         
---  ------            --------------    -----         
 0   Product_Code      1048575 non-null  object        
 1   Warehouse         1048575 non-null  object        
 2   Product_Category  1048575 non-null  object        
 3   Date              1037336 non-null  datetime64[ns]
 4   Order_Demand      1048575 non-null  int64         
dtypes: datetime64[ns](1), int64(1), object(3)
memory usage: 40.0+ MB


### Adding Date, Month, Year, Week, Day of Month, and Day of Week 
May use as features in later analysis

In [22]:
df['Year'] = df.Date.dt.year
df.head(1)

,Product_Code,Warehouse,Product_Category,Date,Order_Demand,Year
0,Product_0993,Whse_J,Category_028,2012-07-27,100,2012.0


In [23]:
df['Month'] = df.Date.dt.month
df.head(1)

,Product_Code,Warehouse,Product_Category,Date,Order_Demand,Year,Month
0,Product_0993,Whse_J,Category_028,2012-07-27,100,2012.0,7.0


In [24]:
df['Week'] = df.Date.dt.week
df.head(1)

,Product_Code,Warehouse,Product_Category,Date,Order_Demand,Year,Month,Week
0,Product_0993,Whse_J,Category_028,2012-07-27,100,2012.0,7.0,30.0


In [25]:
df['DoM'] = df.Date.dt.day
df.head()

,Product_Code,Warehouse,Product_Category,Date,Order_Demand,Year,Month,Week,DoM
0,Product_0993,Whse_J,Category_028,2012-07-27,100,2012.0,7.0,30.0,27.0
1,Product_0979,Whse_J,Category_028,2012-01-19,500,2012.0,1.0,3.0,19.0
2,Product_0979,Whse_J,Category_028,2012-02-03,500,2012.0,2.0,5.0,3.0
3,Product_0979,Whse_J,Category_028,2012-02-09,500,2012.0,2.0,6.0,9.0
4,Product_0979,Whse_J,Category_028,2012-03-02,500,2012.0,3.0,9.0,2.0


In [26]:
df['DoW'] = df.Date.dt.dayofweek
df.head(1)

,Product_Code,Warehouse,Product_Category,Date,Order_Demand,Year,Month,Week,DoM,DoW
0,Product_0993,Whse_J,Category_028,2012-07-27,100,2012.0,7.0,30.0,27.0,4.0


In [27]:
df.shape

(1048575, 10)

In [28]:
df.dtypes

Product_Code                object
Warehouse                   object
Product_Category            object
Date                datetime64[ns]
Order_Demand                 int64
Year                       float64
Month                      float64
Week                       float64
DoM                        float64
DoW                        float64
dtype: object

In [29]:
#Get the lowest and highest dates in the dataset.
df['Date'].min() , df['Date'].max()
#There is data for 6 years. great.

(Timestamp('2011-01-08 00:00:00'), Timestamp('2017-01-09 00:00:00'))

### Simplify Product Codes and Product Category values for later analysis

In [30]:
df['Product_Code'] = df['Product_Code'].map(lambda x: x.lstrip('Product_'))

In [31]:
df.Product_Code.head()

0    0993
1    0979
2    0979
3    0979
4    0979
Name: Product_Code, dtype: object

In [32]:
df['Warehouse'] = df['Warehouse'].map(lambda x: x.lstrip('Warehouse_'))

In [33]:
df['Product_Category'] = df['Product_Category'].map(lambda x: x.lstrip('Category_'))

In [34]:
df.head()

,Product_Code,Warehouse,Product_Category,Date,Order_Demand,Year,Month,Week,DoM,DoW
0,0993,J,028,2012-07-27,100,2012.0,7.0,30.0,27.0,4.0
1,0979,J,028,2012-01-19,500,2012.0,1.0,3.0,19.0,3.0
2,0979,J,028,2012-02-03,500,2012.0,2.0,5.0,3.0,4.0
3,0979,J,028,2012-02-09,500,2012.0,2.0,6.0,9.0,3.0
4,0979,J,028,2012-03-02,500,2012.0,3.0,9.0,2.0,4.0


### Drop 11k observations without dates since only 11 % of observations

In [35]:
df.isnull().sum()

Product_Code            0
Warehouse               0
Product_Category        0
Date                11239
Order_Demand            0
Year                11239
Month               11239
Week                11239
DoM                 11239
DoW                 11239
dtype: int64

In [36]:
df = df.dropna(axis = 0, subset = ['Date'])

In [37]:
df.shape

(1037336, 10)

In [38]:
df.isnull().sum()

Product_Code        0
Warehouse           0
Product_Category    0
Date                0
Order_Demand        0
Year                0
Month               0
Week                0
DoM                 0
DoW                 0
dtype: int64

In [39]:
# Pickle data 
with open('cleaned_product_data.pkl', 'wb') as to_write:
    pickle.dump(df, to_write)

In [40]:
df.dtypes

Product_Code                object
Warehouse                   object
Product_Category            object
Date                datetime64[ns]
Order_Demand                 int64
Year                       float64
Month                      float64
Week                       float64
DoM                        float64
DoW                        float64
dtype: object

### Next step is to visualize data in 02_EDA file